# Generate Spread using dedicated service - F#

### Overview
Spread indicator is commonly used to provide midpoint value from two given inputs. 

Spread is generally impacted by two major factors: 
* *Volatility*: the more volatile an asset, the higher the spread
* *Liquidity*: the more liquid an asset, the lower the spread

When correlated with a range of indicators, it gives an idea about the market quality.

This sample demonstrates how to request and plot the spread from a dedicated data service.

### Inputs/outputs
Spread indicator requires instrument's identifier and date time intervals as per inputs and returns the spread data points.

For each timestamp, a spread data point is the difference between the bid and the ask prices. 

This sample shows how to plot a simple spread graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve top of the book tick data from the hosted service. The queried endpoint in this script are:
* *TickBookService*: to directly retrieve best Bid/Ask tick data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run Spread sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.32.*"
#r "nuget: XPlot.Plotly.Interactive, 4.0.6"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Tick.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Core
open Grpc.Net.Client
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Retrieve data
To request *tick data* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [ ]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Time period delimitation

In [ ]:
// Build the bars request date interval (we are using Google date time format : we have to cast the dates)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = 2021, Month = 03, Day = 05 ),
    EndDate = new Date ( Year = 2021, Month = 03, Day = 05 )
  )

// Build the bars request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
let timeInterval = 
  new TimeInterval(
    StartTime = new TimeOfDay ( Hours = 16, Minutes = 00, Seconds = 00 ),
    EndTime = new TimeOfDay ( Hours = 16, Minutes = 30, Seconds = 00 ) 
  )

#### 3.3 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

In [ ]:
// Generate the tick book request
let request = new TickBookRequest ( Constraints = constraints )
let identifier = new Identifier ( Exchange = exchange, Ticker = ticker)
request.Identifiers.Add([| identifier |]) 

In [ ]:
// Instantiate the tick book service
let service = new TickBookService.TickBookServiceClient(channel)

### Step 4: Visualize data

#### 4.1 Retrieve Spread data

In [ ]:
open FSharp.Control.Tasks.V2
open System.Collections.Generic

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()

In [ ]:
// Get the limits
let limits = new List<BookLimit>()
let call = service.TickBook(request, headers)
let e = call.ResponseStream.ReadAllAsync()

let add (collection : List<BookLimit>) (r : TickBookResponse) =
    if( r |> isNull |> not && r.Limit |> isNull |> not) then
        collection.Add r.Limit

iterAsync e (add limits)

// Display top of the book
// limits |> display

#### 4.2 Plot Spread

Display *spread* over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.
Plot a simple line graphs for *spread* as follows:

In [ ]:
new Scattergl(
    name = "Spread",
    mode = "lines",
    x = (limits |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
    y = (limits |> Seq.map (fun elt -> elt.Ask.Price - elt.Bid.Price))
)
|> Chart.Plot
|> Chart.WithTitle $"Spread for {ticker}"
|> Chart.WithXTitle "Time"
|> Chart.WithYTitle "Price"
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display